In [1]:
import pandas as pd
import numpy as np
import math
from PredictPerovskites import PredictABX3, PredictAABBXX6

In [2]:
# generate some potential single and double perovskites

A_cations = ['Li', 'Sr', 'La']
B_cations = ['Ti', 'Mn', 'Al']
X_anions = ['O']

count = 0
candidates = {}
X1 = X_anions[0]
for A1 in A_cations:
    for A2 in A_cations:
        for B1 in B_cations:
            for B2 in B_cations:
                if (A1 != A2) and (B1 != B2):
                    candidate = ''.join([A1, A2, B1, B2, X1, '6'])
                    tag = 'double_AB'
                elif (A1 != A2):
                    candidate = ''.join([A1, A2, B1, '2', X1, '6'])
                    tag = 'double_A'
                elif (B1 != B2):
                    candidate = ''.join([A1, '2', B1, B2, X1, '6'])
                    tag = 'double_B'
                else:
                    candidate = ''.join([A1, B1, X1, '3'])
                    tag = 'single'
                tmp_dict = {}
                tmp_dict['A1'] = A1
                tmp_dict['A2'] = A2
                tmp_dict['B1'] = B1
                tmp_dict['B2'] = B2
                tmp_dict['X1'] = X1
                tmp_dict['X2'] = X1
                tmp_dict['tag'] = tag
                tmp_dict['formula'] = candidate
                candidates[candidate] = tmp_dict
                count += 1

print('%i candidate formulas' % count)
CCX3s = [k for k in candidates if candidates[k]['tag'] == 'single']
A2BBX6s = [k for k in candidates if candidates[k]['tag'] == 'double_B']
print('%i single perovskite candidates: %s' % (len(CCX3s), CCX3s))
print("%i A2BB'X6 candidates: %s" % (len(A2BBX6s), A2BBX6s))

81 candidate formulas
9 single perovskite candidates: ['LiTiO3', 'LiMnO3', 'LiAlO3', 'SrTiO3', 'SrMnO3', 'SrAlO3', 'LaTiO3', 'LaMnO3', 'LaAlO3']
18 A2BB'X6 candidates: ['Li2TiMnO6', 'Li2TiAlO6', 'Li2MnTiO6', 'Li2MnAlO6', 'Li2AlTiO6', 'Li2AlMnO6', 'Sr2TiMnO6', 'Sr2TiAlO6', 'Sr2MnTiO6', 'Sr2MnAlO6', 'Sr2AlTiO6', 'Sr2AlMnO6', 'La2TiMnO6', 'La2TiAlO6', 'La2MnTiO6', 'La2MnAlO6', 'La2AlTiO6', 'La2AlMnO6']


In [3]:
# if single, assign A/B and classify; if double, use given A/B and classify

properties = ['A', 'B',
              'nA', 'nB', 'nX',
              'rA', 'rB', 'rX',
              't', 't_pred',
              'tau', 'tau_pred']

results = {}
clf = PredictABX3('').calibrate_tau
for candidate in candidates:
    tmp_dict = candidates[candidate]
    A1 = tmp_dict['A1']
    A2 = tmp_dict['A2']
    B1 = tmp_dict['B1']
    B2 = tmp_dict['B2']
    X1 = tmp_dict['X1']
    X2 = tmp_dict['X2']
    classifier = PredictAABBXX6(A1, A2, B1, B2, X1, X2)
    for prop in properties:
        tmp_dict[prop] = getattr(classifier, prop)
    tmp_dict['tau_prob'] = classifier.tau_prob(clf)
    results[candidate] = tmp_dict        
    if not math.isnan(tmp_dict['tau']):
        print('%s classified as %s (tau = %.2f; P = %.2f)' % (candidate, ['perovskite' if tmp_dict['tau_pred'] == 1 else 'nonperovskite'][0],
                                                  tmp_dict['tau'], tmp_dict['tau_prob']))
    else:
        print('%s could not be charge-balanced...' % candidate)

LiTiO3 could not be charge-balanced...
Li2TiMnO6 classified as nonperovskite (tau = 5.07; P = 0.19)
Li2TiAlO6 could not be charge-balanced...
Li2MnTiO6 classified as nonperovskite (tau = 5.07; P = 0.19)
LiMnO3 classified as nonperovskite (tau = 5.96; P = 0.06)
Li2MnAlO6 classified as nonperovskite (tau = 4.82; P = 0.26)
Li2AlTiO6 could not be charge-balanced...
Li2AlMnO6 classified as nonperovskite (tau = 4.82; P = 0.26)
LiAlO3 could not be charge-balanced...
LiSrTi2O6 could not be charge-balanced...
LiSrTiMnO6 classified as nonperovskite (tau = 4.83; P = 0.26)
LiSrTiAlO6 could not be charge-balanced...
LiSrMnTiO6 classified as nonperovskite (tau = 4.83; P = 0.26)
LiSrMn2O6 could not be charge-balanced...
LiSrMnAlO6 classified as nonperovskite (tau = 5.39; P = 0.12)
LiSrAlTiO6 could not be charge-balanced...
LiSrAlMnO6 classified as nonperovskite (tau = 5.39; P = 0.12)
LiSrAl2O6 could not be charge-balanced...
LiLaTi2O6 classified as nonperovskite (tau = 4.26; P = 0.49)
LiLaTiMnO6 clas

In [4]:
# write results to file

with open('classified_formulas.csv', 'w') as f:
    f.write('formula,tag,tau,tau_pred,tau_prob,t,t_pred,A,B,A1,A2,B1,B2,X1,X2,nA,nB,nX,rA,rB,rX\n')
    for r in results:
        d = results[r]
        seq = [d['formula'], d['tag'],
               d['tau'], d['tau_pred'], d['tau_prob'],
               d['t'], d['t_pred'],
               d['A'], d['B'],
               d['A1'], d['A2'], d['B1'], d['B2'], d['X1'], d['X2'],
               d['nA'], d['nB'], d['nX'],
               d['rA'], d['rB'], d['rX']
               ]
        seq = [str(val) for val in seq]
        f.write(','.join(seq)+'\n')


In [5]:
# read file into pandas DataFrame
# NOTE: tau_prob can be > 0.5 with tau_pred == -1 because of the cross-validation used during Platt's scaling

df = pd.read_csv('classified_formulas.csv')

df.head()

,formula,tag,tau,tau_pred,tau_prob,t,t_pred,A,B,A1,...,B1,B2,X1,X2,nA,nB,nX,rA,rB,rX
0,LiTiO3,single,NaN,NaN,NaN,NaN,NaN,Li,Ti,Li,...,Ti,Ti,O,O,NaN,NaN,NaN,NaN,NaN,1.4
1,Li2TiMnO6,double_B,5.068813,-1.0,0.187661,0.896441,-1.0,NaN,NaN,Li,...,Ti,Mn,O,O,1.0,5.0,-2.0,0.92,0.43,1.4
2,Li2TiAlO6,double_B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Li,...,Ti,Al,O,O,NaN,NaN,NaN,NaN,NaN,NaN
3,Li2MnTiO6,double_B,5.068813,-1.0,0.187661,0.896441,-1.0,NaN,NaN,Li,...,Mn,Ti,O,O,1.0,5.0,-2.0,0.92,0.43,1.4
4,LiMnO3,single,5.961561,-1.0,0.055315,0.948259,1.0,Li,Mn,Li,...,Mn,Mn,O,O,1.0,5.0,-2.0,0.92,0.33,1.4


In [6]:
# retrieve data for a given compound

compound = 'LiMnO3'
prop = 'rA'

print('The %s of %s = %.2f' % (prop, compound, df[prop].get((df.formula == compound)).tolist()[0]))

The rA of LiMnO3 = 0.92
